In [1]:
import joblib

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
def get_data():
    df = pd.read_csv('data/winemag-data_first150k.csv', index_col=0)
    top_20_varieties = df.value_counts(df.variety).index[:20]
    df = df[df.variety.isin(top_20_varieties)]
    df = df.fillna(df.mode().iloc[0])
    cols = ['country', 'description', 'designation', 'province', 'region_1', 'variety']
    df = df[cols]
    return df

full_df = get_data()
cols = full_df.columns

In [3]:
sample = full_df.sample(1000)
X = sample.drop(columns=['variety']).apply(' '.join, axis=1)
y = sample.variety
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
def train(X,y, text_clf=None):
    if not text_clf:
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', RandomForestClassifier())
        ])
    text_clf.fit(X, y)
    def eval(text_clf, y_test):
        y_pred = text_clf.predict(y_test)
        score = metrics.roc_auc_score(y_test, y_pred)
        print(f'ROC AUC: {score:.3f}')
        return score
    joblib.dump(text_clf, 'model.joblib')
    return text_clf
    
text_clf = train(X_train, y_train)

/c/Users/samhi/Documents/projects/wine-reviews/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/c/Users/samhi/Documents/projects/wine-reviews/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/c/Users/samhi/Documents/projects/wine-reviews/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

                               precision    recall  f1-score   support

                    Red Blend       1.00      1.00      1.00        15
                   Sangiovese       0.00      0.00      0.00        26
                    Zinfandel       0.39      1.00      0.56        25
                     Riesling       0.00      0.00      0.00         3
                   Chardonnay       0.00      0.00      0.00         7
                   Pinot Noir       1.00      1.00      1.00         8
                         Rosé       0.00      0.00      0.00         6
                       Merlot       1.00      1.00      1.00        25
                        Syrah       0.00      0.00      0.00         4
     Bordeaux-style Red Blend       0.33      1.00      0.49        16
           Cabernet Sauvignon       1.00      1.00      1.00         9
                  White Blend       1.00      1.00      1.00         7
                     Nebbiolo       0.00      0.00      0.00         6
     

In [181]:
second_sample = df[~df.index.isin(sample.index)].sample(1000)
X = second_sample[cols].apply(' '.join, axis=1)
y = second_sample.variety
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [22]:
pipeline = joblib.load('model.joblib')
new_data = df.loc[1:3,cols].drop(columns=['variety'])

def predict(df):
    df_ = df.apply(' '.join, axis=1)
    return pipeline.predict(df_)

predictions = predict(new_data)
predictions


array(['Chardonnay', 'Pinot Noir'], dtype=object)

In [ ]:
df.to_json(orient='records')

In [41]:
json_data

'["Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white. Dark gold in color, it layers toasted hazelnut, pear compote and orange peel flavors, reveling in the succulence of its 122 g\\/L of residual sugar.","This spent 20 months in 30% new French oak, and incorporates fruit from Ponzi\'s Aurora, Abetina and Madrona vineyards, among others. Aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. Tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. Drink now through 2032."]'

In [43]:
def predict_api(df):
    payload = df.to_json(orient='records')
    headers = {'content-type': 'application/json'}
    res = requests.post("http://localhost:5000/score", data=payload, headers=headers)
    predictions = res.json()['prediction']
    df['prediction'] = predictions
    return df

pred_data = predict_api(new_data)

AttributeError: 'Series' object has no attribute 'split'

In [185]:
pred_data

,country,description,designation,province,region_1,prediction
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,California,Knights Valley,Pinot Noir
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,Oregon,Willamette Valley,Cabernet Sauvignon


## Using Online Learning

from river import compose
from river import preprocessing
from 